<a href="https://colab.research.google.com/github/74obunga/SQL-Notebook-SpaceX-Rockets/blob/main/SQL_Notebook_SpaceX_Rockets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas==1.3.3 sqlalchemy sqlite3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 21.6 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

# Load the dataset
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Create an SQLite engine
engine = create_engine('sqlite:///spacex.db')

# Save the dataframe to a SQL table
df.to_sql("SPACEXTBL", engine, if_exists='replace', index=False, method="multi")

# Create an SQLite connection
conn = sqlite3.connect('spacex.db')


In [ ]:
query1 = """
SELECT DISTINCT "Launch_Site"
FROM SPACEXTBL
"""

unique_launch_sites = pd.read_sql_query(query1, conn)
print(unique_launch_sites)


    Launch_Site
0   CCAFS LC-40
1   VAFB SLC-4E
2    KSC LC-39A
3  CCAFS SLC-40


In [ ]:
query2 = """
SELECT *
FROM SPACEXTBL
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5
"""
records_cca = pd.read_sql_query(query2, conn)
print(records_cca)


         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

In [ ]:
query3 = """
SELECT SUM("Payload_Mass__kg_")
FROM SPACEXTBL
WHERE "Customer" = 'NASA (CRS)'
"""
total_payload_mass = pd.read_sql_query(query3, conn)
print(total_payload_mass)


   SUM("Payload_Mass__kg_")
0                     45596


In [ ]:
query4 = """
SELECT AVG("Payload_Mass__kg_")
FROM SPACEXTBL
WHERE "Booster_Version" = 'F9 v1.1'
"""
average_payload_mass = pd.read_sql_query(query4, conn)
print(average_payload_mass)


   AVG("Payload_Mass__kg_")
0                    2928.4


In [ ]:
query5 = """
SELECT MIN(Date)
FROM SPACEXTBL
WHERE "Landing_Outcome" = 'Success (ground pad)'
"""
first_successful_landing = pd.read_sql_query(query5, conn)
print(first_successful_landing)


    MIN(Date)
0  2015-12-22


In [ ]:
query6 = """
SELECT "Booster_Version"
FROM SPACEXTBL
WHERE "Landing_Outcome" = 'Success (drone ship)'
AND "Payload_Mass__kg_" > 4000 AND "Payload_Mass__kg_" < 6000
"""
boosters_success_drone_ship = pd.read_sql_query(query6, conn)
print(boosters_success_drone_ship)


  Booster_Version
0     F9 FT B1022
1     F9 FT B1026
2  F9 FT  B1021.2
3  F9 FT  B1031.2


In [ ]:
query7 = """
SELECT "Landing_Outcome", COUNT(*)
FROM SPACEXTBL
GROUP BY "Landing_Outcome"
"""
mission_outcomes = pd.read_sql_query(query7, conn)
print(mission_outcomes)


           Landing_Outcome  COUNT(*)
0       Controlled (ocean)         5
1                  Failure         3
2     Failure (drone ship)         5
3      Failure (parachute)         2
4               No attempt        21
5              No attempt          1
6   Precluded (drone ship)         1
7                  Success        38
8     Success (drone ship)        14
9     Success (ground pad)         9
10    Uncontrolled (ocean)         2


In [ ]:
query8 = """
SELECT "Booster_Version"
FROM SPACEXTBL
WHERE "Payload_Mass__kg_" = (
    SELECT MAX("Payload_Mass__kg_")
    FROM SPACEXTBL
)
"""
boosters_max_payload = pd.read_sql_query(query8, conn)
print(boosters_max_payload)


   Booster_Version
0    F9 B5 B1048.4
1    F9 B5 B1049.4
2    F9 B5 B1051.3
3    F9 B5 B1056.4
4    F9 B5 B1048.5
5    F9 B5 B1051.4
6    F9 B5 B1049.5
7   F9 B5 B1060.2 
8   F9 B5 B1058.3 
9    F9 B5 B1051.6
10   F9 B5 B1060.3
11  F9 B5 B1049.7 


In [ ]:
query9 = """
SELECT SUBSTR(Date, 6, 2) AS month, "Landing_Outcome", "Booster_Version", "Launch_Site"
FROM SPACEXTBL
WHERE "Landing_Outcome" LIKE 'Failure (drone ship)'
AND SUBSTR(Date, 1, 4) = '2015'
"""
failure_2015 = pd.read_sql_query(query9, conn)
print(failure_2015)


  month       Landing_Outcome Booster_Version  Launch_Site
0    01  Failure (drone ship)   F9 v1.1 B1012  CCAFS LC-40
1    04  Failure (drone ship)   F9 v1.1 B1015  CCAFS LC-40


In [ ]:
query10 = """
SELECT "Landing_Outcome", COUNT(*) as outcome_count
FROM SPACEXTBL
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome"
ORDER BY outcome_count DESC
"""
ranked_outcomes = pd.read_sql_query(query10, conn)
print(ranked_outcomes)


          Landing_Outcome  outcome_count
0              No attempt             10
1    Success (drone ship)              5
2    Failure (drone ship)              5
3    Success (ground pad)              3
4      Controlled (ocean)              3
5    Uncontrolled (ocean)              2
6     Failure (parachute)              2
7  Precluded (drone ship)              1
